In [35]:
from selenium import webdriver
import requests
import time
import pandas as pd
from IPython.display import clear_output

In [36]:
# 웹드라이브 실행
driver = webdriver.Chrome('chromedriver')

In [37]:
# 다음 영화 사이트를 들어간다.
site = 'https://movie.daum.net/premovie/theater'
time.sleep(1)
driver.get(site)

In [38]:
# 스크롤 한다
for idx in range(10) :
    time.sleep(2)
    script = 'window.scrollTo(0, document.body.scrollHeight)'
    driver.execute_script(script)

In [39]:
# 영화 전체 목록을 가져온다.
a1 = driver.find_element_by_css_selector('#mainContent > div > div.box_movie > ol')
a1

<selenium.webdriver.remote.webelement.WebElement (session="4c7e223201c2f39204e0e676bddcaf99", element="4609f4d5-215e-4a19-83e6-a58fb6b27f6a")>

In [40]:
# 영화 전체 목록에서 li 태그들을 가져온다.
li_list = a1.find_elements_by_css_selector('li')
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="4c7e223201c2f39204e0e676bddcaf99", element="40b6babb-4105-47fd-a293-6de8266d1ad2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c7e223201c2f39204e0e676bddcaf99", element="720c67af-e4c2-41ed-a60b-f54ffb8c3944")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c7e223201c2f39204e0e676bddcaf99", element="1f9759c9-8837-40e4-9dbf-70d1cde086e7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c7e223201c2f39204e0e676bddcaf99", element="2692fb75-4550-44af-b25e-2e2c9b8f8a71")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c7e223201c2f39204e0e676bddcaf99", element="bfe9e1da-0ce9-4048-9fb2-4a21140e99ea")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c7e223201c2f39204e0e676bddcaf99", element="a9302f76-8d5b-4579-83e4-73746a947364")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c7e223201c2f39204e0e676bddcaf99", element="3fb60cdf-1825-45f0-8535-d6

In [42]:
# 데이터를 담을 딕셔너리
data_dict = {
    '제목' : [],
    '예매율' : [],
    '평점' : [],
    '개봉일' : []
}

# 영화의 수 만큼 반복한다
for movie_tag in li_list :
    # 영화 제목을 가져온다
    title_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > strong > a')
    title = title_tag.text.strip()

    # 평점
    rating_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(1) > span')
    rating = rating_tag.text.strip()

    # 예매율
    ticket_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(2) > span')
    ticket = ticket_tag.text.strip().replace('%','')

    # 개봉일
    open_date_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_info > span')
    open_date = open_date_tag.text.strip()

    # 영화 포스터
    poster_tag = movie_tag.find_element_by_css_selector('div > div.thumb_item > div.poster_movie > img')
    # src 속성의 값을 가져온다
    src_attr = poster_tag.get_attribute('src')

    # 영화의 제목을 파일명으로 사용할 것이기 때문에
    # os에서 거부하는 파일명을 정제한다
    char_list = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']

    file_name = title
    for c1 in char_list :
        file_name = file_name.replace(c1, ' ')

    # 이미지 데이터를 내려받는다
    img_res = requests.get(src_attr)

    # 저장한다
    with open(f'poster/{file_name}.jpg', 'wb') as fp :
        fp.write(img_res.content)

    # print(title)
    # print(rating)
    # print(ticket)
    # print(open_date)
    # print('-------------------------')

    # 데이터를 담는다
    data_dict['제목'].append(title)
    data_dict['예매율'].append(ticket)
    data_dict['평점'].append(rating)
    data_dict['개봉일'].append(open_date)
    
# 데이터 프레임 생성
df1 = pd.DataFrame(data_dict)

# 저장
df1.to_csv('daum_movie.csv', encoding='utf-8-sig', index = False)
print('저장완료')

저장완료
